# 3.3 GDAL and OGR

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#3.3-GDAL-and-OGR" data-toc-modified-id="3.3-GDAL-and-OGR-3.1">3.3 GDAL and OGR</a></span><ul class="toc-item"><li><span><a href="#3.3.1-The-MODIS-data" data-toc-modified-id="3.3.1-The-MODIS-data-3.1.1">3.3.1 The MODIS data</a></span><ul class="toc-item"><li><span><a href="#3.3.1.1-File-Naming-Convention" data-toc-modified-id="3.3.1.1-File-Naming-Convention-3.1.1.1">3.3.1.1 File Naming Convention</a></span></li><li><span><a href="#3.3.1.2-Dataset-Naming-Convention" data-toc-modified-id="3.3.1.2-Dataset-Naming-Convention-3.1.1.2">3.3.1.2 Dataset Naming Convention</a></span></li></ul></li><li><span><a href="#3.3.2-MODIS-dataset-access" data-toc-modified-id="3.3.2-MODIS-dataset-access-3.1.2">3.3.2 MODIS dataset access</a></span><ul class="toc-item"><li><span><a href="#3.3.2.1-gdal.ReadAsArray()" data-toc-modified-id="3.3.2.1-gdal.ReadAsArray()-3.1.2.1">3.3.2.1 <code>gdal.ReadAsArray()</code></a></span></li><li><span><a href="#3.3.2.2-Metadata" data-toc-modified-id="3.3.2.2-Metadata-3.1.2.2">3.3.2.2 Metadata</a></span></li></ul></li><li><span><a href="#3.3.3-Reading-and-displaying-data" data-toc-modified-id="3.3.3-Reading-and-displaying-data-3.1.3">3.3.3 Reading and displaying data</a></span><ul class="toc-item"><li><span><a href="#3.3.3.1-glob" data-toc-modified-id="3.3.3.1-glob-3.1.3.1">3.3.3.1 <code>glob</code></a></span></li><li><span><a href="#3.3.3.2-reading-and-displaying-image-data" data-toc-modified-id="3.3.3.2-reading-and-displaying-image-data-3.1.3.2">3.3.3.2 reading and displaying image data</a></span></li><li><span><a href="#3.3.3.3-subplot-plotting" data-toc-modified-id="3.3.3.3-subplot-plotting-3.1.3.3">3.3.3.3 subplot plotting</a></span></li><li><span><a href="#3.3.3.3-tile-stitching" data-toc-modified-id="3.3.3.3-tile-stitching-3.1.3.4">3.3.3.3 tile stitching</a></span></li><li><span><a href="#3.3.3.4-gdal-virtual-file" data-toc-modified-id="3.3.3.4-gdal-virtual-file-3.1.3.5">3.3.3.4 <code>gdal</code> virtual file</a></span></li></ul></li><li><span><a href="#The-country-borders-dataset" data-toc-modified-id="The-country-borders-dataset-3.1.4">The country borders dataset</a></span></li><li><span><a href="#Putting-it-all-together" data-toc-modified-id="Putting-it-all-together-3.1.5">Putting it all together</a></span></li><li><span><a href="#Exercises" data-toc-modified-id="Exercises-3.1.6">Exercises</a></span></li></ul></li></ul></div>




In this section, we'll look at combining both raster and vector data to provide a masked dataset ready to use. We will produce a combined dataset of leaf area index (LAI) over the UK derived from the MODIS sensor. The MODIS LAI product is produced every 4 days and it is provided spatially tiled. Each tile covers around 1200 km x 1200 km of the Earth's surface. Below you can see a map showing the MODIS tiling convention.



## 3.3.1 The MODIS LAI data

You should by now be able to download MODIS data, but in this case, the data are provided (or downloaded for you) in the `data` folder as files `MCD15A3H.A2018273.h17v03.006.2018278143630.hdf`  and `MCD15A3H.A2018273.h18v03.006.2018278143633.hdf` by running the code below.


In [ ]:
from geog0111.geog_data import *

filenames = ['MCD15A3H.A2018273.h17v03.006.2018278143630.hdf', \
            'MCD15A3H.A2018273.h18v03.006.2018278143633.hdf']
destination_folder="data"

for file_name in filenames:
    f = procure_dataset(file_name,verbose=True,\
                        destination_folder=destination_folder)
    print(file_name,f)

We want to select the LAI layers, so let's have a look at the contents ('sub datasets') of one of the files.

To do this with `gdal`:

* make the full filename (folder name, plus the filename in that folder). Use `Path` for this, but convert to a string.
* open the file, store as `g`
* get the list `g.GetSubDatasets()` and loop over this

In [ ]:
import gdal
from pathlib import Path
from geog0111.geog_data import *

filenames = ['MCD15A3H.A2018273.h17v03.006.2018278143630.hdf', \
            'MCD15A3H.A2018273.h18v03.006.2018278143633.hdf']
destination_folder="data"

for file_name in filenames:
    # form full filename as a string
    # and print with an underline of = 
    file_name = Path(destination_folder).joinpath(file_name).as_posix()
    print(file_name)
    print('='*len(file_name))
    
    # open the file as g
    g = gdal.Open(file_name)
    # loop over the subdatasets
    for d in g.GetSubDatasets():
        print(d)

So we see that the data is in `HDF4` format, and that it has a number of layers. The dataset/layer we're interested in

`HDF4_EOS:EOS_GRID:"data/MCD15A3H.A2018273.h18v03.006.2018278143633.hdf":MOD_Grid_MCD15A3H:Lai_500m`.


### 3.3.1.1 File Naming Convention

This section taken from [NASA MODIS product page](https://nsidc.org/data/mod10a1).

Example File Name:

`data/MOD10A1.A2000055.h15v01.006.2016061160800.hdf`



`FOLDER/MOD[PID].A[YYYY][DDD].h[NN]v[NN].[VVV].[yyyy][ddd][hhmmss].hdf`

Refer to Table 3.3.1 for descriptions of the file name variables listed above.



|  Variable | Description  |  
|---|---|
| FOLDER| folder/directory name of file|
| MOD  |  MODIS/Terra  (`MCD` means combined)| 
|  PID |   Product ID|  
| A	|Acquisition date follows|
|YYYY	|Acquisition year|
|DDD	|Acquisition day of year|
|h[NN]v[NN]	|Horizontal tile number and vertical tile number (see Grid for details.)|
|VVV	|Version (Collection) number|
|yyyy	|Production year|
|ddd	|Production day of year|
|hhmmss	|Production hour/minute/second in GMT|
|.hdf	|HDF-EOS formatted data file|
Table 3.3.1. Variables in the MODIS File Naming Convention

![](https://nsidc.org/sites/nsidc.org/files/images/modis-sin-grid.png)


### 3.3.1.2 Dataset Naming Convention

Example Dataset Name:

`HDF4_EOS:EOS_GRID:"data/MCD15A3H.A2018273.h18v03.006.2018278143633.hdf":MOD_Grid_MCD15A3H:Lai_500m`


`FORMAT:"FILENAME":MOD_Grid_PRODUCT:LAYER`

|  Variable | Description  |  
|---|---|
|FORMAT| file format, `HDF4_EOS:EOS_GRID`|
|FILENAME| dataset file name, see below|
|PRODUCT| MODIS product code e.g. `MCD15A3H`|
|LAYER| sub-dataset name e.g. `Lai_500m`|
Table 3.3.2. Variables in the MODIS Dataset Naming Convention


**Exercise E3.3.1**

* Check you're happy that the other datasets (e.g. `LaiStdDev_500m`) follow the same convention as `Lai_500m`
* work out what the dataset/layer name would be for the dataset product `MOD10A1` version `6` for the $1^{st}$ January 2018, for tile `h25v06` for the layer `NDSI_Snow_Cover`. You will find product information [on the relevant NASA page](https://nsidc.org/data/mod10a1). You may not be able to access the production date/time, but just put a placeholder for that now.
* phrase the filename and layer name as '`f`' strings, e.g. starting `f'HDF4_EOS:EOS_GRID:"{filename}":MOD_Grid_{}'` etc.

**Hint**:

You can explore the filenames by looking into the [Earthdata link](https://n5eil01u.ecs.nsidc.org/MOSA/).

![](https://n5eil01u.ecs.nsidc.org/MOSA/MYD10A1.006/2018.01.01/BROWSE.MYD10A1.A2018001.h25v05.006.2018003025825.1.jpg)

In [ ]:
# do exercise here

## 3.3.2 MODIS dataset access

### 3.3.2.1 `gdal.ReadAsArray()`

We can now access the dataset names and open the datasets in `gdal` directly, e.g.:

`HDF4_EOS:EOS_GRID:"data/MCD15A3H.A2018273.h18v03.006.2018278143633.hdf":MOD_Grid_MCD15A3H:Lai_500m`

We can read the dataset with `g.ReadAsArray()`, after we have opened it. It returns a numpy array.

In [ ]:
import gdal
import numpy as np

filename = 'data/MCD15A3H.A2018273.h17v03.006.2018278143630.hdf'
dataset_name = f'HDF4_EOS:EOS_GRID:"{filename:s}":MOD_Grid_MCD15A3H:Lai_500m'
print(f"dataset: {dataset_name}")

g = gdal.Open(dataset_name)
data = g.ReadAsArray()

print(type(data))
print('max:',data.max())
print('max:',data.min())
# get unique values, for interst
print('unique values:',np.unique(data))

**Exercise E3.3.2**

* print out some further summary statistics of the dataset
* print out the data type and `shape`
* how many rows and columns does the dataset have?

In [ ]:
# do exercise here

### 3.3.2.2 Metadata

There will generally be a set of metadata associated with a geospatial dataset. This will describe e.g. the processing chain, special codes in the dataset, and projection and other information.

In `gdal`, w access the metedata using `g.GetMetadata()`. A dictionary is returned.

In [ ]:
filename = 'data/MCD15A3H.A2018273.h17v03.006.2018278143630.hdf'
dataset_name = f'HDF4_EOS:EOS_GRID:"{filename:s}":MOD_Grid_MCD15A3H:Lai_500m'
g = gdal.Open(dataset_name)

print ("\nMetedata Keys:\n")
# get the metadata dictionary keys
for k in g.GetMetadata().keys():
    print(k)

Let's look at some of these metadata fields:

In [ ]:
import gdal
import numpy as np

filename = 'data/MCD15A3H.A2018273.h17v03.006.2018278143630.hdf'
dataset_name = f'HDF4_EOS:EOS_GRID:"{filename:s}":MOD_Grid_MCD15A3H:Lai_500m'
print(f"dataset: {dataset_name}")

g = gdal.Open(dataset_name)
# get the metadata dictionary keys
for k in ["LONGNAME","CHARACTERISTICBINSIZE500M",\
          "MOD15A2_FILLVALUE_DOC",\
          "GRINGPOINTLATITUDE.1","GRINGPOINTLONGITUDE.1",\
          'scale_factor']:
    print(k,g.GetMetadata()[k])

So we see that the datasets use the MODIS Sinusoidal projection. Also we see that the pixel spacing is around 463m, there is a scale factor of 0.1 to be applied etc.

**Exercise E3.3.3**

look at the metadata to discover:

* the number of rows and columns in the dataset
* the range of valid values

In [ ]:
# do exercise here

## 3.3.3 Reading and displaying data

### 3.3.3.1 `glob`

Let us now suppose that we want to examine an `hdf` file that we have previously downloaded and stored in the directiory `data`.

How can we get a view into this directory to the the names of the files there?

The answer to this is `glob`, which we can access from the `pathlib` module.

Let's look in the `data` directory:

In [ ]:
from pathlib import Path

# look in this directory
in_directory = Path('data')

filenames = in_directory.glob('*')
print('files in the directory',in_directory,':')
for f in filenames:
    print(f.name)

We use the argument `'data/*'` where `*` is a wildcard. Any filenames that match this pattern will be returned as a list.

If we want the list sorted, we need to use the `sorted()` method. This is similar to the list `sort` we have seen previously, but returns the sorted list.

The wildcard `*` here means a match to zero or more characters, so this is matching all names in the directory `data`. The wildcard `**` would mean [all files here and all sub-directories](https://docs.python.org/3/library/pathlib.html#pathlib.Path.glob).



We could be more subtle with this, e.g. matching only files ending `hdf`:

In [ ]:
from pathlib import Path

filenames = sorted(Path('data').glob('*'))

for f in filenames:
    print(f.name)

**Exercise 3.3.4**

* adapt the code above to return only hdf filenames for the tile `h18v03`

In [ ]:
# do exercise here

### 3.3.3.2 reading and displaying image data

Let's now read some data as above.

we do this with:

    g.Open(gdal_fname)
    data = g.ReadAsArray()
    
Originally the data are `uint8` (unsigned 8 bit data), but we need to multiply them by `scale_factor` (0.1 here) to convert to physical units. This also casts the data type to `float`.

We can straightforwardly plot the images using `matplotlib`. We first importt the library:

    import matplotlib.pylab as plt
    
Then set up the figure size:

    plt.figure(figsize=(10,10))
    
Plot the image:

    plt.imshow( data, vmin=0, vmax=6,cmap=plt.cm.inferno_r)
    
where here `data` is a 2-D dataset. We can set limits to the image scaling (`vmin`, `vmax`), so that we emphasise a particular range of values, and we can apply custom colourmaps (`cmap=plt.cm.inferno_r`).

Finally here, we set a title, and plot a colour wedge to show the data scale. The `scale=0.8` here allows us to align the size of the scale with the plotted image size.

    plt.title(dataset_name)
    plt.colorbar(shrink=0.8)
    
If we want to save the plotted image to a file, e.g. in the directory `images`, we use:

    plt.savefig(out_filename)
    
    

In [ ]:
import gdal
from pathlib import Path
import matplotlib.pylab as plt

# get only v03 hdf names
filenames = sorted(Path('data').glob('*v03*.hdf'))


out_directory = Path('images')

for filename in filenames:
    # pull the tile name from the filename
    # to use as plot title
    tile = filename.name.split('.')[2]
    
    dataset_name = f'HDF4_EOS:EOS_GRID:"{str(filename):s}\":MOD_Grid_MCD15A3H:Lai_500m'
    g = gdal.Open(dataset_name)
    data = g.ReadAsArray()
    scale_factor = float(g.GetMetadata()['scale_factor'])
    
    print(dataset_name,scale_factor)
    print('*'*len(dataset_name))
    print(type(data),data.dtype,data.shape,'\n')
    
    data = data * scale_factor
    print(type(data),data.dtype,data.shape,'\n')
    plt.figure(figsize=(10,10))
    plt.imshow( data, vmin=0, vmax=6,cmap=plt.cm.inferno_r)
    plt.title(tile)
    plt.colorbar(shrink=0.8)
    
    # save figure as png
    plot_name = filename.stem + '.png'
    print(plot_name)
    out_filename = out_directory.joinpath(plot_name)
    plt.savefig(out_filename)

In [ ]:
# Let's check the images we saved are there!
# and access some file info while we are here
# using pathlib
from pathlib import Path
from datetime import datetime

for f in Path('images').glob('MCD*v03*.png'):
    
    # get the file size in bytes 
    size_in_B = f.stat().st_size
    
    # get the file modification time (ns)
    mod_date_ns = f.stat().st_mtime_ns
    mod_date = datetime.fromtimestamp(mod_date_ns // 1000000000)
    
    print(f'{f} {size_in_B} Bytes {mod_date}')

### 3.3.3.3 subplot plotting

Often, we want to have several figures on the same plot. We can do this with `plt.subplots()`:

The way we set the title and other features is slightly diifferent, but there are many example of different plot types on the web we can follow as examples.

In [ ]:
import gdal
from pathlib import Path
import matplotlib.pylab as plt

filenames = sorted(Path('data').glob('*v03*.hdf'))

out_directory = Path('images')

'''
Set up subplots of 1 row x 2 columns
'''
fig, axs = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True,
                       figsize=(10,5))
# need to force axs collapse to a 2D array
# for indexing to be easy T here is transpose
# to get row/col the right way around
axs = np.array(axs).T.flatten()

for i,filename in enumerate(filenames):
    # pull the tile name from the filename
    # to use as plot title
    tile = filename.name.split('.')[2]
    
    
    dataset_name = f'HDF4_EOS:EOS_GRID:"{str(filename):s}\":MOD_Grid_MCD15A3H:Lai_500m'
    g = gdal.Open(dataset_name)
    data = g.ReadAsArray() * float(g.GetMetadata()['scale_factor'])

    img = axs[i].imshow(data, interpolation="nearest", vmin=0, vmax=4,
                 cmap=plt.cm.inferno_r)
    axs[i].set_title(tile)
    plt.colorbar(img,ax=axs[i],shrink=0.7)
    
# save figure as pdf this time
plot_name = 'joinedup.pdf'
print(plot_name)
out_filename = out_directory.joinpath(plot_name)
plt.savefig(out_filename)

**Exercise 3.3.5**

* Use the code in [3.3.1](#3.3.1-The-MODIS-data) to get the datasets for the tiles `h17v04` and `h18v04`:

    MCD15A3H.A2018273.h17v04.006.2018278143630.hdf	
    MCD15A3H.A2018273.h18v04.006.2018278143638.hdf

* use subplot as above to plot a 2x2 set of subplots of these data.


**Hint**

The code should look much like that above, but you need to give the fiuller list of filenames and set the subplot shape.

The result should look like:

![](images/joinedup4.pdf)

In [ ]:
# do exercise here

### 3.3.3.3 tile stitching

You may want to generate a single view of the 4 tiles.

We could achieve this by stitching things together "by hand"...

**recipe:**

* First, lets generate a 3D dataset with all 4 tiles, so we have the images stored as members of a list `data[0]`,`data[1]`,`data[2]` and `data[3]`:

        data = []
        for filename in filenames:
            dataname = f'HDF4_EOS:EOS_GRID:"{str(filename):s}":MOD_Grid_MCD15A3H:Lai_500m'
            g = gdal.Open(dataname)
            data.append(g.ReadAsArray() * scale)

* then, we produce vertical stacks of the first two and last two files. This can be done in various ways, but it is perhaps clearest to use `np.vstack()`

        top = np.vstack([data[0],data[1]])
        bot = np.vstack([data[2],data[3]])
        
* then, produce a horizontal stack of these stacks:

        lai_stich = np.hstack([top,bot])
        
and plot the dataset

In [ ]:
import gdal
from pathlib import Path
import matplotlib.pylab as plt

scale = 0.1

filenames = sorted(Path('data').glob('*v0*.hdf'))

data = []
for filename in filenames:
    dataname = f'HDF4_EOS:EOS_GRID:"{str(filename)}":MOD_Grid_MCD15A3H:Lai_500m'
    g = gdal.Open(dataname)
    # append each image to the data list
    data.append(g.ReadAsArray() * scale)

top = np.vstack([data[0],data[1]])
bot = np.vstack([data[2],data[3]])

lai_stich = np.hstack([top,bot])

plt.figure(figsize=(10,10))
plt.imshow(lai_stich, interpolation="nearest", vmin=0, vmax=4,
          cmap=plt.cm.inferno_r)
plt.colorbar(shrink=0.8)

**Exercise 3.3.6**

* examine how the `vstack` and `hstack` methods work. Print out the shape of the array after stacking to appreciate this.
* how big (in pixels) is the whole dataset now? 
* If a `float` is 64 bits, how many bytes is this data array likely to be?

In [ ]:
# do exercise here

### 3.3.3.4 `gdal` virtual file

However, stitching in this way is problematic if you want to mosaic many tiles, as you need to read in all the data in memory. Also,some tiles may be missing. GDAL allows you to create a mosaic as [virtual file format](https://www.gdal.org/gdal_vrttut.html), using gdal.BuildVRT (check the documentation). 

This function takes two inputs: the output filename (`stitch_up.vrt`) and a set of GDAL format filenames. It returns the open output dataset, so that we can check what it looks like with e.g. `gdal.Info`

In [ ]:
import gdal
from pathlib import Path

# need to convert filenames to strings
# which we can do with p.as_posix() or str(p)
filenames = sorted([p.as_posix() for p in Path('data').glob('*v0*.hdf')])
    
stitch_vrt = gdal.BuildVRT("stitch_up.vrt", filenames)

print(gdal.Info(stitch_vrt))

So we see that we now have 4800 columns by 4800 rows dataset, centered around 0 degrees North, 0 degrees W. Let's plot the data...

In [ ]:
stitch_vrt.GetSubDatasets()

In [ ]:
# stitch_vrt is an already opened GDAL dataset, needs to be read in
plt.imshow(stitch_vrt.ReadAsArray()*stitch_vrt.Scale,
           interpolation="nearest", vmin=0, vmax=4, 
          cmap=plt.cm.inferno_r)

## The country borders dataset

A number of vectors with countries and administrative subdivisions are available. The [TM_WORLD_BORDERS shapefile](http://thematicmapping.org/downloads/TM_WORLD_BORDERS-0.3.zip) is popular and in the public domain. You can see it, and have a look at the data [here](https://koordinates.com/layer/7354-tm-world-borders-03/). We need to download and unzip this file... We'll use requests as before, and we'll unpack the zip file using [`shutil.unpack_archive`](https://docs.python.org/3/library/shutil.html#shutil.unpack_archive)

We recall from the metadata in `valid_range` that the range of valid values in the dataset is 0 to 100. `MOD15A2_FILLVALUE_DOC` shows that numbers `249` to `254` mean that the land cover class implies no LAI. The `_Fillvalue`  is `255` here. 

We generally need to consider carefully what we want to do about fill values (i.e. no reading) or data for which we supect the quality is poor. This will often involve some form of iterpolation.

In [ ]:
k = "MOD15A2_FILLVALUE_DOC"
print(k,g.GetMetadata()[k])

k = 'valid_range'
print(k,g.GetMetadata()[k])

In [ ]:
import requests
import shutil 

tm_borders_url = "http://thematicmapping.org/downloads/TM_WORLD_BORDERS-0.3.zip"

r = requests.get(tm_borders_url)
with open("data/TM_WORLD_BORDERS-0.3.zip", 'wb') as fp:
    fp.write (r.content)

shutil.unpack_archive("data/TM_WORLD_BORDERS-0.3.zip",
                     extract_dir="data/")

Make sure you have the relevant files available in your `data` folder! We can then inspect the dataset using the command line tool `ogrinfo`. We can call it from the shell by appending the `!` symbol, and select that we want to check only the data for the UK (stored in the `FIPS` field with value `UK`):

<div class="alert alert-block alert-info">
  It is worth noting that using OGR's queries trying to match a string, the string needs to be surrounded by <pre>'</pre>. You can also use more complicated SQL queries if you wanted to.
</div>

In [ ]:
!ogrinfo -al data/TM_WORLD_BORDERS-0.3.shp 

We inmediately see that the coordinates for the UK are in several polygons, and in WGS84 (Latitude and Longitude in decimal degrees). This is incompatible with the MODIS data.

We can use GDAL to quickly apply the vector feature for the UK as a mask. There are several ways of doing this, but the simplest is to use [gdal.Warp](https://www.gdal.org/gdalwarp.html) (the link is to the command line tool). In this case, we just want to create:

* an in-memory (i.e. not saved to a file) dataset. We can use the format `MEM`, so no file is written out
* where the FIPS field is equal to "UK", we want the LAI to show, elsewhere, we set it to (e.g.) "no data" (e.g. -999)


The mosaicked version of the MODIS LAI product is in file `stitch_up.vrt`. Since we're not saving the output to a file (`MEM` output option), we can leave the output as an empty string `""`. The shapefile comes with the `cutline` options:

* `cutlineDSName` that's the name of the vector file we want to use as a cutline
* `cutlineWhere` that's the selection statement for the attribute table in the dataset. 

To set the no data value to 200, we can use the option `dstNodata=200`. This is because very large values in the LAI product are already indicated to be invalid.

We can then just very quickly perform this and check...

In [ ]:
g = gdal.Warp("", "stitch_up.vrt",
         format = 'MEM',dstNodata=200,
          cutlineDSName = 'data/TM_WORLD_BORDERS-0.3.shp', cutlineWhere = "FIPS='UK'")
masked_lai = g.ReadAsArray()/10.
plt.imshow(masked_lai, interpolation="nearest", vmin=0, vmax=6, 
          cmap=plt.cm.inferno_r)

So that works as expected, but since we haven't actually told GDAL anything about the output (other than apply the mask), we still have a 4800 pixel wide dataset. You may want to crop it by looking for where the original dataset is  *not* -999, or in fact, for pixels that are bigger than 0. You'll be pleased to know that this is a great slicing application!

In [ ]:
lai = g.ReadAsArray()
mask = np.nonzero(lai < 20)
min_y = mask[0].min()
max_y = mask[0].max() + 1

min_x = mask[1].min()
max_x = mask[1].max() + 1

plt.figure(figsize=(12,12))
plt.imshow( lai[min_y:max_y,
               min_x:max_x]/10., vmin=0, vmax=6,
           cmap=plt.cm.inferno_r)
plt.colorbar()


## Putting it all together

The previous steps can be put together in a function. It is even possible to simplify things a bit more by not even storing the name of the stitiched up VRT file:

## Exercises

* Put everything together as a function
* Mask pixels outside the Netherlands
* Mask pixels outside Ireland


In [ ]:
def mosaic_and_mask_data(gdal_fnames, vector_file, vector_where):
    stitch_vrt = gdal.BuildVRT("", gdal_fnames)
    g = gdal.Warp("", stitch_vrt,
                 format = 'MEM', dstNodata=200,
                  cutlineDSName = vector_file,
                  cutlineWhere = vector_where)

    return g

g = mosaic_and_mask_data(gdal_fnames, "data/TM_WORLD_BORDERS-0.3.shp",
                         "FIPS='UK'")
lai = g.ReadAsArray()
mask = np.nonzero(lai < 200)
min_y = mask[0].min()
max_y = mask[0].max() + 1

min_x = mask[1].min()
max_x = mask[1].max() + 1

plt.figure(figsize=(12,12))
plt.imshow( lai[min_y:max_y,
               min_x:max_x]/10., vmin=0, vmax=6,
           cmap=plt.cm.inferno_r)
plt.colorbar()


In [ ]:
g = mosaic_and_mask_data(gdal_fnames, "data/TM_WORLD_BORDERS-0.3.shp",
                         "FIPS='NL'")
lai = g.ReadAsArray()
mask = np.nonzero(lai < 200)
min_y = mask[0].min()
max_y = mask[0].max() + 1

min_x = mask[1].min()
max_x = mask[1].max() + 1

plt.figure(figsize=(12,12))
plt.imshow( lai[min_y:max_y,
               min_x:max_x]/10., vmin=0, vmax=6,
           cmap=plt.cm.inferno_r)
plt.colorbar()